Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0

# Set up Financial Fraud dataloader, model training and inference 

## Table of Contents
1. Load processed graph data (in notebook 1.1) into a data dict for the data loader for model training 
* The data dictionary is defined in the referenced TADDY modeling framework to easily fetch relevant data during training 
2. load the model training and data sampling configurations 
* Eigenvalue decomposition based on the adjacency matrix is used for node sampling. Nodes are sampled across multiple snapshots for the edge of interest based on a defined time window. 
3. pass the data dict in step (1) to the model 
4. train the model 
5. apply model inference on the specific snapshot

In [1]:
import sys 
import os

In [2]:
sys.path.append('../../')
sys.path.append('../../src/')

In [3]:
import pickle
import numpy as np
import torch
import yaml
from anomaly_detection_spatial_temporal_data.model.model_config import TaddyConfig
from anomaly_detection_spatial_temporal_data.utils import ensure_directory
from anomaly_detection_spatial_temporal_data.model.dynamic_graph import Taddy

/home/ec2-user/SageMaker/external-repos/anomaly-detection-spatial-temporal-data-workshop/src/kedro-taddy-venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load processed graph data in notebook 1.1

In [4]:
with open("../../data/03_primary/financial_fraud/training_data.pkl", 'rb') as file:
    data = pickle.load(file)

In [5]:
rows, cols, labels, weights, headtail, train_size, test_size, nb_nodes, nb_edges = data

In [10]:
degrees = np.array([len(x) for x in headtail])
num_snap = test_size + train_size
labels = [torch.LongTensor(label) for label in labels]

snap_train = list(range(num_snap))[:train_size]
snap_test = list(range(num_snap))[train_size:]

In [14]:
idx = list(range(nb_nodes))
index_id_map = {i:i for i in idx}
idx = np.array(idx)

# Set model training and data sampling configuration and create data dictionary

### load the model training hyperparameters

In [15]:
train_config_file = '../../conf/base/parameters/taddy.yml'

with open(train_config_file, "r") as stream:
    try:
        train_config=yaml.safe_load(stream)
        print(train_config)
    except yaml.YAMLError as exc:
        print(exc)

{'data_load_options': {'c': 0.15, 'eps': 0.001, 'random_state': 3, 'batch_size': 256, 'load_all_tag': False, 'neighbor_num': 5, 'window_size': 2, 'compute_s': True, 'eigen_file_name': 'data/05_model_input/eigen.pkl'}, 'model_options': {'neighbor_num': 5, 'window_size': 2, 'batch_size': 256, 'embedding_dim': 32, 'num_hidden_layers': 2, 'num_attention_heads': 2, 'seed': 1, 'print_feq': 10, 'lr': 0.001, 'weight_decay': '5e-4', 'max_epoch': 10, 'spy_tag': True, 'max_hop_dis_index': 100, 'max_inti_pos_index': 100, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.5, 'attention_probs_dropout_prob': 0.3, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12, 'is_decoder': False, 'save_directory': 'data/07_model_output/'}, 'infer_options': {'snap_num': 6}}


### load the data sampling parameters
   * window size is the number of snapshots looked back during node sampling 
   * neighbor number is the number of neighbors to sample close to the source and target nodes of the edge of interest 

In [16]:
eigen_file_name = "../../data/05_model_input/financial_fraud/eigen_tmp.pkl"
data_loader_config = train_config['data_load_options']

data_loader_config

{'c': 0.15,
 'eps': 0.001,
 'random_state': 3,
 'batch_size': 256,
 'load_all_tag': False,
 'neighbor_num': 5,
 'window_size': 2,
 'compute_s': True,
 'eigen_file_name': 'data/05_model_input/eigen.pkl'}

### Define relevant functions to node sampling, the purpose of eah function is explained in brief docstring.  

In [17]:
import scipy.sparse as sp
from numpy.linalg import inv

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix. (0226)"""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def adj_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -0.5).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx).dot(r_mat_inv)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation. (0226)"""
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    # adj_np = np.array(adj.todense())
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj_normalized = sparse_mx_to_torch_sparse_tensor(adj_normalized)
    return adj_normalized

def get_adjs(rows, cols, weights, nb_nodes, eigen_file_name, data_loader_config):
    """Generate adjacency matrix and conduct eigenvalue decomposition for node sampling"""
    if not os.path.exists(eigen_file_name):
        generate_eigen = True
        print('Generating eigen as: ' + eigen_file_name)
    else:
        generate_eigen = False
        print('Loading eigen from: ' + eigen_file_name)
        with open(eigen_file_name, 'rb') as f:
            eigen_adjs_sparse = pickle.load(f)
        eigen_adjs = []
        for eigen_adj_sparse in eigen_adjs_sparse:
            eigen_adjs.append(np.array(eigen_adj_sparse.todense()))

    adjs = []
    if generate_eigen:
        eigen_adjs = []
        eigen_adjs_sparse = []

    for i in range(len(rows)):
        adj = sp.csr_matrix((weights[i], (rows[i], cols[i])), shape=(nb_nodes, nb_nodes), dtype=np.float32)
        adjs.append(preprocess_adj(adj))
        if data_loader_config['compute_s']:
            if generate_eigen:
                eigen_adj = data_loader_config['c'] * inv((sp.eye(adj.shape[0]) - (1 - data_loader_config['c']) * adj_normalize(adj)).toarray())
                for p in range(adj.shape[0]):
                    eigen_adj[p,p] = 0.
                eigen_adj = normalize(eigen_adj)
                eigen_adjs.append(eigen_adj)
                eigen_adjs_sparse.append(sp.csr_matrix(eigen_adj))

        else:
            eigen_adjs.append(None)

    if generate_eigen:
        with open(eigen_file_name, 'wb') as f:
            pickle.dump(eigen_adjs_sparse, f, pickle.HIGHEST_PROTOCOL)

    return adjs, eigen_adjs

In [18]:
ensure_directory(eigen_file_name)
edges = [np.vstack((rows[i], cols[i])).T for i in range(num_snap)]
adjs, eigen_adjs = get_adjs(rows, cols, weights, nb_nodes, eigen_file_name, data_loader_config)

Loading eigen from: ../../data/05_model_input/financial_fraud/eigen_tmp.pkl


### The data dictionary defined in TADDY modeling framework 
  * X is the node feature matrix (We did not generate node feature for this use case. Hence we are aiming to learn from the graph structural information and its evloving pattern) 
  * A is the adjacency matrix (a popular way to represent a graph)
  * S is the eigen decomposition result of A 
  * degrees stores all the node degrees
  * other keys are self-explanatory: edges store edge list, y is the edge label, snap_train are snapshots for training and snap_test are snapshots for testing. num_snap is the total number of snapshots. 

In [19]:
data_dict = {
    'X': None, 
    'A': adjs, 
    'S': eigen_adjs, 
    'index_id_map': index_id_map, 
    'edges': edges,
    'y': labels, 
    'idx': idx, 
    'snap_train': snap_train, 
    'degrees': degrees,
    'snap_test': snap_test, 
    'num_snap': num_snap}

In [20]:
train_config

{'data_load_options': {'c': 0.15,
  'eps': 0.001,
  'random_state': 3,
  'batch_size': 256,
  'load_all_tag': False,
  'neighbor_num': 5,
  'window_size': 2,
  'compute_s': True,
  'eigen_file_name': 'data/05_model_input/eigen.pkl'},
 'model_options': {'neighbor_num': 5,
  'window_size': 2,
  'batch_size': 256,
  'embedding_dim': 32,
  'num_hidden_layers': 2,
  'num_attention_heads': 2,
  'seed': 1,
  'print_feq': 10,
  'lr': 0.001,
  'weight_decay': '5e-4',
  'max_epoch': 10,
  'spy_tag': True,
  'max_hop_dis_index': 100,
  'max_inti_pos_index': 100,
  'hidden_act': 'gelu',
  'hidden_dropout_prob': 0.5,
  'attention_probs_dropout_prob': 0.3,
  'initializer_range': 0.02,
  'layer_norm_eps': 1e-12,
  'is_decoder': False,
  'save_directory': 'data/07_model_output/'},
 'infer_options': {'snap_num': 6}}

In [21]:
#change save path for notebook
train_config['model_options']['save_directory'] = '../../data/07_model_output/financial_fraud' 

if not os.path.exists(train_config['model_options']['save_directory']):
    os.makedirs(train_config['model_options']['save_directory'])

In [22]:
model_config = TaddyConfig(config=train_config['model_options'])
model_obj = Taddy(data_dict, model_config)

model_config.save_directory

'../../data/07_model_output/financial_fraud'

# Train model

In [23]:
learned_result,save_model_path = model_obj.run()

Epoch: 1, loss:0.6880, Time: 25.7884s
Epoch: 2, loss:0.6785, Time: 26.1162s
Epoch: 3, loss:0.6700, Time: 26.8577s
Epoch: 4, loss:0.6623, Time: 27.2026s
Epoch: 5, loss:0.6460, Time: 25.9393s
Epoch: 6, loss:0.6000, Time: 27.8746s
Epoch: 7, loss:0.5732, Time: 26.4647s
Epoch: 8, loss:0.5414, Time: 25.7860s
Epoch: 9, loss:0.4494, Time: 27.2630s
Epoch: 10, loss:0.4204, Time: 25.6965s
Snap: 05 | AUC: 0.6745
Snap: 06 | AUC: 0.6489
Snap: 07 | AUC: 0.6310
Snap: 08 | AUC: 0.6447
Snap: 09 | AUC: 0.8219
TOTAL AUC:0.6386


# Model training result 

In [24]:
learned_result

{1: {'train_loss': 0.6880353391170502},
 2: {'train_loss': 0.678493544459343},
 3: {'train_loss': 0.6700391620397568},
 4: {'train_loss': 0.6623023152351379},
 5: {'train_loss': 0.6460123509168625},
 6: {'train_loss': 0.6000367254018784},
 7: {'train_loss': 0.5731981694698334},
 8: {'train_loss': 0.5413680970668793},
 9: {'train_loss': 0.4493589624762535},
 10: {'train_loss': 0.42043692618608475, 'test_auc': 0.6385655578120397}}

In [25]:
save_model_path

'../../data/07_model_output/financial_fraud/taddy_model_9.pth'

# Run inference on the specific snapshot 

### load trained model 

In [26]:
import torch
import transformers

In [27]:
model = torch.load(save_model_path)

In [28]:
snap_num = 9

In [29]:
pred = model.predict(snap_num)

Generating embeddings...
Embeddings created!


In [30]:
# pred

In [31]:
# labels[1]

In [32]:
from sklearn import metrics

auc = metrics.roc_auc_score(labels[snap_num],pred)

auc

0.8218944980147476

# References

Edgar Alonso Lopez-Rojas and Stefan Axelsson. 2014. BANKSIM: A BANK PAYMENTS SIMULATOR FOR FRAUD DETECTION RESEARCH.

Yixin Liu, Shirui Pan, Yu Guang Wang, Fei Xiong, Liang Wang, Qingfeng Chen, and Vincent CS Lee. 2015. Anomaly Detection in Dynamic Graphs via Transformer.